## Sentiment Prediction on IMDB Movie Reviews

This notebook demonstrates how to load a pre-trained Simple RNN model to predict the sentiment of movie reviews from the IMDB dataset. I have also added custom functions to enhance the usability of the model and provide detailed explanations of each step.

### 1. Import Libraries and Load the Model

First, we import the necessary libraries and load the pre-trained model. This model was trained on the IMDB dataset to classify movie reviews as either positive or negative.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

We then load the IMDB dataset's word index to help decode the reviews and understand the model's predictions.

In [2]:
# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

Next, we load the pre-trained model, which uses ReLU activation in its Simple RNN layers.

In [3]:
# Load the pre-trained model
model = load_model('model.h5')
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 500, 128)          32896     
                                                                 
 dropout (Dropout)           (None, 500, 128)          0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1325313 (5.06 MB)
Trainable params: 1325313 (5.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


To inspect the model's inner workings, we print out the model's weights.

In [4]:
model.get_weights()

[array([[-0.01785068,  0.01257522, -0.01849201, ...,  0.02737954,
         -0.01658337, -0.03896516],
        [ 0.02769358,  0.00831363, -0.00283951, ...,  0.03523619,
         -0.0181964 , -0.12319023],
        [ 0.04864456,  0.04688917, -0.01911613, ..., -0.0015069 ,
         -0.06152027, -0.12863229],
        ...,
        [ 0.0210665 ,  0.00178189,  0.0389583 , ...,  0.04459306,
          0.03170855,  0.10278009],
        [-0.0106249 , -0.01080296, -0.04215729, ..., -0.02182666,
         -0.01483092, -0.08489985],
        [ 0.07182481, -0.0186534 , -0.0415456 , ..., -0.08701115,
          0.04192985, -0.03757128]], dtype=float32),
 array([[ 0.06201936, -0.10811633,  0.00183683, ...,  0.11658924,
         -0.23910615, -0.0692889 ],
        [ 0.01075747,  0.02303515, -0.00803249, ...,  0.04664249,
         -0.04821857, -0.03791673],
        [-0.04392743, -0.04314497,  0.04237111, ...,  0.068992  ,
          0.10616311, -0.1198985 ],
        ...,
        [-0.14973275, -0.03107561, -0.0

### 2. Helper Functions
To make the model more user-friendly, I created helper functions for decoding reviews and preprocessing user input.

The first function decodes encoded reviews back into readable text.

In [5]:
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

The second function preprocesses text input by encoding the words and padding the sequence to match the model's input requirements.

In [6]:
# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

### 3. Prediction Function
I have written a function that uses the preprocessed input to predict the sentiment of a review. The function returns both the sentiment (Positive/Negative) and the prediction score.

In [7]:
# Prediction function
def predict_sentiment(review):
    preprocessed_input = preprocess_text(review)
    prediction = model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment, prediction[0][0]

### 4. User Input and Prediction
Now, let's test the prediction function with an example movie review. The review's sentiment and prediction score will be printed.

In [8]:
# Example positive review for prediction
example_review = "Fantastic movie! Great acting and a thrilling plot kept me hooked the entire time. Highly recommended!"

sentiment, score = predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 [==============================] - 1s 875ms/step
Review: Fantastic movie! Great acting and a thrilling plot kept me hooked the entire time. Highly recommended!
Sentiment: Positive
Prediction Score: 0.998945951461792


In [9]:
# Example negative  review for prediction
example_review = "Disappointing film with uneven acting and a predictable plot. It failed to engage and left me underwhelmed."

sentiment, score = predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 [==============================] - 0s 80ms/step
Review: Disappointing film with uneven acting and a predictable plot. It failed to engage and left me underwhelmed.
Sentiment: Negative
Prediction Score: 0.04452497884631157
